# Using_an_SBML_model (python3)

This is a copy of Using_an_SBML_model (Python2) but built with a Python3 kernel to make sure everything works.

# Using an SBML model

## Getting started

### Installing libraries

Before you start, you will need to install a couple of libraries:
   
The [ModelSeedDatabase](https://github.com/ModelSEED/ModelSEEDDatabase) has all the biochemistry we'll need. You can install that with `git clone`.
   
The [PyFBA](http://linsalrob.github.io/PyFBA) library has detailed [installation instructions](http://linsalrob.github.io/PyFBA/installation.html). Don't be scared, its mostly just `pip install`.

(Optional) Also, get the [SEED Servers](https://github.com/linsalrob/SEED_Servers_Python) as you can get a lot of information from them. You can install the git python repo from github.  Make sure that the SEED_Servers_Python is in your PYTHONPATH.

We start with importing some modules that we are going to use. 

We import *sys* so that we can use standard out and standard error if we have some error messages.<br>
We import *copy* so that we can make a deep copy of data structures for later comparisons.<br>
Then we import the *PyFBA* module to get started.

In [2]:
import sys
import os
import copy
import PyFBA

We are using /home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.0-py3.9.egg/PyFBA/Biochemistry/ModelSEEDDatabase for our data


## Running an SBML model

If you have run your genome through RAST, you can download the [SBML](http://www.sbml.org/) model and use that directly.

We have provided an [SBML model of *Citrobacter sedlakii*](https://raw.githubusercontent.com/linsalrob/PyFBA/master/example_data/Citrobacter/Citrobacter_sedlakii.sbml) that you can download and use. You can right-ctrl click on this link and save the SBML file in the same location you are running this iPython notebook.

We use this SBML model to demonstrate the key points of the FBA approach: defining the reactions, including the boundary, or drainflux, reactions; the compounds, including the drain compounds; the media; and the reaction bounds. 

We'll take it step by step!

We start by parsing the model:

In [3]:
sbml = PyFBA.parse.parse_sbml_file("../example_data/Citrobacter/Citrobacter_sedlakii.sbml")

We are logging to /home/redwards/GitHubsLinux/PyFBA/iPythonNotebooks/PyFBA.2021-06-01T08:54:03.924916.log


### Find all the reactions and identify those that are boundary reactions

We need a set of reactions to run in the model. In this case, we are going to run all the reactions in our SBML file. However, you can change this set if you want to knock out reactions, add reactions, or generally modify the model. We store those in the `reactions_to_run` set.

The boundary reactions refer to compounds that are secreted but then need to be removed from the `reactions_to_run` set. We usually include a consumption of those compounds that is open ended, as if they are draining away. We store those reactions in the `uptake_secretion_reactions` dictionary.


In [4]:
# Get a dict of reactions.
# The key is the reaction ID, and the value is a metabolism.reaction.Reaction object
reactions = sbml.reactions
reactions_to_run = set()
uptake_secretion_reactions = {}
biomass_equation = None
for r in reactions:
    if 'biomass_equation' == r:
        biomass_equation = reactions[r]
        print(f"Our biomass equation is {biomass_equation.readable_name}")
        continue
    is_boundary = False
    for c in reactions[r].all_compounds():
        if c.uptake_secretion:
            is_boundary = True
            break
    if is_boundary:
        reactions[r].is_uptake_secretion = True
        uptake_secretion_reactions[r] = reactions[r]
    else:
        reactions_to_run.add(r)


Our biomass equation is Citrobacter_sedlakii_119_auto_biomass


At this point, we can take a look at how many reactions are in the model, not counting the biomass reaction:

In [5]:
print(f"The biomass equation is {biomass_equation}")
print("There are {} reactions in the model".format(len(reactions)))
print("There are {}".format(len(uptake_secretion_reactions)),
      "uptake/secretion reactions in the model")
print("There are {}".format(len(reactions_to_run)),
      "reactions to be run in the model")

The biomass equation is biomass_equation: Citrobacter_sedlakii_119_auto_biomass
There are 1574 reactions in the model
There are 0 uptake/secretion reactions in the model
There are 1573 reactions to be run in the model


In [6]:
with open('rgood.txt', 'w') as out:
    for r in reactions:
        out.write(f"{r}\n")

In [7]:
biomass_equation.equation

' (0.00778132482043096) cpd00063: Ca2 (location: c) +  (0.352889948968272) cpd00156: L_Valine (location: e) +  (0.00778132482043096) cpd00030: Mn2 (location: e) +  (0.00778132482043096) cpd00205: K (location: c) +  (0.428732289454499) cpd00035: L_Alanine (location: e) +  (0.128039715997337) cpd00060: L_Methionine (location: e) +  (0.15480760087483) cpd00066: L_Phenylalanine (location: c) +  (0.00778132482043096) cpd00017: S_Adenosyl_L_methionine (location: c) +  (0.00778132482043096) cpd00010: CoA (location: c) +  (0.0609084652443221) cpd15665: Peptidoglycan_polymer_n_subunits (location: c) +  (0.0841036156544863) cpd00052: CTP (location: c) +  (0.00778132482043096) cpd10516: fe3 (location: e) +  (0.01468498342018) cpd00357: TTP (location: c) +  (0.00778132482043096) cpd00099: Cl_ (location: e) +  (0.01468498342018) cpd00356: dCTP (location: c) +  (0.00778132482043096) cpd10515: Fe2 (location: e) +  (0.00778132482043096) cpd00254: Mg (location: c) +  (0.242249358141304) cpd00322: L_Iso

In [9]:
for l in biomass_equation.left_compounds:
    print(f"'{l}' : {biomass_equation.get_left_compound_abundance(l)}, ", end=" ")

'cpd00066: L_Phenylalanine (location: c)' : 0.15480760087483,  'cpd00016: Pyridoxal_phosphate (location: c)' : 0.00778132482043096,  'cpd00132: L_Asparagine (location: e)' : 0.201205267995816,  'cpd00156: L_Valine (location: e)' : 0.352889948968272,  'cpd00099: Cl_ (location: e)' : 0.00778132482043096,  'cpd00038: GTP (location: c)' : 0.135406821203723,  'cpd00003: NAD (location: c)' : 0.00778132482043096,  'cpd17041: Protein_biosynthesis (location: c)' : 1.0,  'cpd00033: Glycine (location: e)' : 0.510820469745475,  'cpd00322: L_Isoleucine (location: e)' : 0.242249358141304,  'cpd00254: Mg (location: c)' : 0.00778132482043096,  'cpd17043: RNA_transcription (location: c)' : 1.0,  'cpd00048: Sulfate (location: e)' : 0.00778132482043096,  'cpd10515: Fe2 (location: e)' : 0.00778132482043096,  'cpd02229: Bactoprenyl_diphosphate (location: c)' : 0.0609084652443221,  'cpd11493: ACP (location: c)' : 0.00778132482043096,  'cpd00161: L_Threonine (location: e)' : 0.211466290532188,  'cpd00006: NA

In [10]:
for l in biomass_equation.right_compounds:
    print(f"'{l}' : {biomass_equation.get_right_compound_abundance(l)}, ", end=" ")

'cpd00067: H (location: e)' : 40.0,  'cpd00012: PPi (location: e)' : 0.405833094852252,  'cpd00008: ADP (location: c)' : 40.0,  'cpd11416: Biomass (location: c)' : 1.0,  'cpd12370: apo_ACP (location: c)' : 0.00778132482043096,  'cpd00009: Phosphate (location: e)' : 39.9922186751796,  'cpd15666: Peptidoglycan_polymer_n_1_subunits (location: c)' : 0.0609084652443221,  

### Find all the compounds in the model, and filter out those that are secreted

We need to filter out uptake and secretion compounds from our list of all compounds before we can make a stoichiometric matrix.

In [7]:
# Get a dict of compounds. 
# The key is the string representation of the compound and
# the value is a metabolite.compound.Compound object
# Get a dict of compounds. 
# The key is the string representation of the compound and
# the value is a metabolite.compound.Compound object
all_compounds = sbml.compounds
# Filter for compounds that are boundary compounds
filtered_compounds = set()
for c in all_compounds:
    if not c.uptake_secretion:
        filtered_compounds.add(c)

Again, we can see how many compounds there are in the model.

In [8]:
print("There are {} total compounds in the model".format(len(all_compounds)))
print("There are {}".format(len(filtered_compounds)),
      "compounds that are not involved in uptake and secretion")

There are 1475 total compounds in the model
There are 1301 compounds that are not involved in uptake and secretion


And now we have the size of our stoichiometric matrix! Notice that the stoichiometric matrix is composed of the reactions that we are going to run and the compounds that are in those reactions (but not the uptake/secretion reactions and compounds).

In [9]:
print(f"The stoichiometric matrix will be {len(reactions_to_run):,} reactions by {len(filtered_compounds):,} compounds")

The stoichiometric matrix will be 1,573 reactions by 1,301 compounds


### Read the media file, and correct the media names

In our [media](https://github.com/linsalrob/PyFBA/tree/master/media) directory, we have a lot of different media formulations, most of which we use with the Genotype-Phenotype project. For this example, we are going to use Lysogeny Broth (LB). There are many different formulations of LB, but we have included the recipe created by the folks at Argonne so that it is comparable with their analysis. You can download [ArgonneLB.txt](https://raw.githubusercontent.com/linsalrob/PyFBA/master/media/ArgonneLB.txt) and put it in the same directory as this iPython notebook to run it.

Once we have read the file we need to correct the names in the compounds. Sometimes when compound names are exported to the SBML file they are modified slightly. This just corrects those names.

In [10]:
# Read the media file
media = PyFBA.parse.read_media_file("/home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/Biochemistry/media/ArgonneLB.txt")
# Correct the names
media = sbml.correct_media(media)
print(f"The media has {len(media)} compounds")

The media has 65 compounds


Checking media compounds: Our compounds do not include  Molybdate
Checking media compounds: Our compounds do not include  Ni2+
Checking media compounds: Our compounds do not include  L-Cystine
Checking media compounds: Our compounds do not include  chromate
Checking media compounds: Our compounds do not include  Thiamine phosphate
Checking media compounds: Our compounds do not include  Vitamin B12
Please note: The warnings about media not being found in compounds are not fatal.
It just means that we did not find that compound anywhere in the reactions, and so it is unlikely to be
needed or used. We typically see a few of these in rich media.


In [11]:
print(f"There are {len(filtered_compounds):,} filtered compounds, {len(all_compounds):,} all compounds, and {len(reactions):,} reactions")

There are 1,301 filtered compounds, 1,475 all compounds, and 1,574 reactions


### Set the reaction bounds for uptake/secretion compounds

The uptake and secretion compounds typically have reaction bounds that allow them to be consumed (i.e. diffuse away from the cell) but not produced. However, our media components can also increase in concentration (i.e. diffuse to the cell) and thus the bounds are set higher. Whenever you change the growth media, you also need to adjust the reaction bounds to ensure that the media can be consumed!


In [12]:
# Adjust the lower bounds of uptake secretion reactions
# for things that are not in the media
for u in uptake_secretion_reactions:
    is_media_component = False
    for c in uptake_secretion_reactions[u].all_compounds():
        if c in media:
            is_media_component = True
    if not is_media_component:
        reactions[u].lower_bound = 0.0
        uptake_secretion_reactions[u].lower_bound = 0.0

### Run the FBA

Now that we have constructed our model, we can run the FBA!

In [13]:
status, value, growth = PyFBA.fba.run_fba(filtered_compounds, reactions,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


In the model there are : 1163 compounds and 1758 reactions
We are loading 1163 rows and 1758 columns


The FBA completed with a flux value of 486.54008183384263 --> growth: True


Length of the media: 65
Number of reactions to run: 1573
Number of compounds in SM: 1163
Number of reactions in SM: 1758
Revised number of total reactions: 1758
Number of total compounds: 1319
SMat dimensions: 1163 x 1758


# IGNORE ALL OF THIS

It is Rob trying to figure out what he broke!


In [14]:
without_ex = set()
for r in reactions_to_run:
    if not r.startswith('EX'):
        without_ex.add(r)
without_ex

{'rxn02305',
 'rxn03137',
 'rxn08306',
 'rxn05555',
 'rxn11268',
 'rxn05594',
 'rxn00791',
 'rxn00799',
 'rxn09450',
 'rxn05064',
 'rxn04113',
 'rxn10212',
 'rxn05565',
 'rxn00856',
 'rxn01396',
 'rxn01620',
 'rxn01506',
 'rxn03537',
 'rxn02339',
 'rxn01255',
 'rxn01275',
 'rxn00006',
 'rxn00237',
 'rxn03397',
 'rxn07587',
 'rxn00211',
 'rxn01807',
 'rxn01858',
 'rxn10233',
 'rxn05164',
 'rxn00691',
 'rxn04928',
 'rxn05175',
 'rxn08468',
 'rxn05429',
 'rxn05335',
 'rxn00762',
 'rxn05199',
 'rxn05433',
 'rxn05467',
 'rxn05247',
 'rxn05937',
 'rxn01548',
 'rxn00692',
 'rxn05177',
 'rxn00634',
 'rxn02504',
 'rxn01674',
 'rxn05649',
 'rxn01018',
 'rxn05485',
 'rxn12643',
 'rxn10180',
 'rxn05560',
 'rxn01390',
 'rxn09104',
 'rxn00950',
 'rxn00224',
 'rxn00717',
 'rxn05552',
 'rxn00606',
 'rxn00297',
 'rxn05448',
 'rxn02934',
 'rxn08550',
 'rxn01314',
 'rxn05464',
 'rxn02988',
 'rxn09143',
 'rxn09111',
 'rxn10904',
 'rxn12851',
 'rxn00193',
 'rxn02429',
 'rxn00242',
 'rxn00184',
 'rxn01520',

In [15]:
with open('rwex.txt', 'w') as out:
    for r in without_ex:
        out.write(f"{r}\n")

In [16]:
len(without_ex)

1399

In [17]:
test_biomass_equation = PyFBA.metabolism.biomass_equation('gram_negative')
test_biomass_equation.equation

'(0.00778132482043) ACP + (40.1101757365) ATP + (0.0609084652443) Bactoprenyl diphosphate + (0.0841036156545) CTP + (0.00778132482043) Ca2+ + (0.00778132482043) Cl- + (0.00778132482043) Co2+ + (0.00778132482043) CoA + (0.00778132482043) Cu2+ + (1) DNA replication + (0.00778132482043) FAD + (0.00778132482043) Fe2+ + (0.135406821204) GTP + (0.510820469745) Glycine + (35.5386858538) H2O + (0.00778132482043) K+ + (0.428732289454) L-Alanine + (0.247156803702) L-Arginine + (0.201205267996) L-Asparagine + (0.201205267996) L-Aspartate + (0.0762884719009) L-Cysteine + (0.219496655995) L-Glutamate + (0.219496655995) L-Glutamine + (0.0794113918032) L-Histidine + (0.242249358141) L-Isoleucine + (0.376088782529) L-Leucine + (0.285970236775) L-Lysine + (0.128039715997) L-Methionine + (0.154807600875) L-Phenylalanine + (0.184698405655) L-Proline + (0.179790960094) L-Serine + (0.211466290532) L-Threonine + (0.0472899299502) L-Tryptophan + (0.115101904973) L-Tyrosine + (0.352889948968) L-Valine + (0.00

In [18]:
status, value, growth = PyFBA.fba.run_fba(all_compounds, reactions,
                                          without_ex, media, test_biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


In the model there are : 1222 compounds and 1584 reactions
We are loading 1222 rows and 1584 columns


The FBA completed with a flux value of 0.0 --> growth: False


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1222
Number of reactions in SM: 1584
Revised number of total reactions: 1758
Number of total compounds: 1553
SMat dimensions: 1222 x 1584


In [19]:
status, value, growth = PyFBA.fba.run_fba(all_compounds, reactions,
                                          without_ex, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


In the model there are : 1163 compounds and 1584 reactions
We are loading 1163 rows and 1584 columns


The FBA completed with a flux value of 402.6089941082853 --> growth: True


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1163
Number of reactions in SM: 1584
Revised number of total reactions: 1758
Number of total compounds: 1553
SMat dimensions: 1163 x 1584


In [26]:
for c in test_biomass_equation.left_compounds:
    print(f"'{c}' : {test_biomass_equation.get_left_compound_abundance(c)}")

'rctn49: GTP (location: c)' : 0.135406821204
'rctn47: fe3 (location: c)' : 0.00778132482043
'rctn26: L-Alanine (location: c)' : 0.428732289454
'rctn23: dGTP (location: c)' : 0.0146849834202
'rctn24: L-Leucine (location: c)' : 0.376088782529
'rctn17: ACP (location: c)' : 0.00778132482043
'rctn21: Pyridoxal phosphate (location: c)' : 0.00778132482043
'rctn31: L-Isoleucine (location: c)' : 0.242249358141
'rctn0: K+ (location: c)' : 0.00778132482043
'rctn33: L-Tryptophan (location: c)' : 0.0472899299502
'rctn5: RNA transcription (location: c)' : 1.0
'rctn45: Mn2+ (location: c)' : 0.00778132482043
'rctn8: Protein biosynthesis (location: c)' : 1.0
'rctn32: L-Glutamate (location: c)' : 0.219496655995
'rctn3: Sulfate (location: c)' : 0.00778132482043
'rctn37: Glycine (location: c)' : 0.510820469745
'rctn44: L-Tyrosine (location: c)' : 0.115101904973
'rctn48: L-Glutamine (location: c)' : 0.219496655995
'rctn43: dCTP (location: c)' : 0.0146849834202
'rctn7: L-Serine (location: c)' : 0.1797909600

In [27]:
for c in test_biomass_equation.right_compounds:
    print(f"'{c}' : {test_biomass_equation.get_right_compound_abundance(c)}")

'rctn52: Peptidoglycan polymer (n-1 subunits) (location: c)' : 0.0609084652443
'rctn52: Phosphate (location: c)' : 39.9922186752
'rctn52: apo-ACP (location: c)' : 0.00778132482043
'rctn52: ADP (location: c)' : 40.0
'rctn52: PPi (location: c)' : 0.405833094852
'rctn52: H+ (location: c)' : 40.0
'rctn52: Biomass (location: c)' : 1.0


In [21]:
for c in biomass_equation.right_compounds:
    print(f"'{c}' : {biomass_equation.get_right_compound_abundance(c)}")

'cpd00008: ADP (location: c)' : 40.0
'cpd12370: apo_ACP (location: c)' : 0.00778132482043096
'cpd11416: Biomass (location: c)' : 1.0
'cpd00009: Phosphate (location: e)' : 39.9922186751796
'cpd15666: Peptidoglycan_polymer_n_1_subunits (location: c)' : 0.0609084652443221
'cpd00012: PPi (location: e)' : 0.405833094852252
'cpd00067: H (location: e)' : 40.0


In [54]:
new_bm = PyFBA.metabolism.Reaction("new biomass equation", 'new biomass')

left_source = test_biomass_equation
#left_source = biomass_equation
# right_source = test_biomass_equation
right_source = biomass_equation

new_bm.add_left_compounds(left_source.left_compounds)
for c in new_bm.left_compounds:
    new_bm.set_left_compound_abundance(c, left_source.get_left_compound_abundance(c))
new_bm.add_right_compounds(right_source.right_compounds)
for c in new_bm.right_compounds:
    new_bm.set_right_compound_abundance(c, right_source.get_right_compound_abundance(c))
new_bm.equation = test_biomass_equation.equation

In [55]:
status, value, growth = PyFBA.fba.run_fba(all_compounds, reactions,
                                          without_ex, media, new_bm,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In the model there are : 1216 compounds and 1584 reactions
We are loading 1216 rows and 1584 columns


The FBA completed with a flux value of 0.0 --> growth: False


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1216
Number of reactions in SM: 1584
Revised number of total reactions: 1758
Number of total compounds: 1553
SMat dimensions: 1216 x 1584


In [58]:
tbel = {}
testplus = copy.copy(new_bm)
for c in testplus.left_compounds:
    tbel[c.name] = c

for c in biomass_equation.left_compounds:
    if c.name in tbel:
        if tbel[c.name].location != c.location:
            print(f"{c.name}: {c.location} -> {tbel[c.name].location}")
        else:
            print(f"{c.name} is missing")
            testplus.add_left_compounds({c})
            testplus.set_left_compound_abundance(c, biomass_equation.get_left_compound_abundance(c))



FAD is missing
CoA is missing
TTP is missing
fe3: e -> c
ACP is missing
NADP is missing
NAD is missing
H2O: e -> c
Riboflavin: e -> c
CTP is missing
Mg is missing
dATP is missing
ATP is missing
dGTP is missing
UTP is missing
GTP is missing
dCTP is missing
Glycine: e -> c
Sulfate: e -> c


In [ ]:
for c in biomass_equation.left_compounds:
    if c.name in tbel:
        if tbel[c.name].location != c.location:
            print(f"{c.name}: {c.location} -> {tbel[c.name].location}")
            testplus.

In [57]:
status, value, growth = PyFBA.fba.run_fba(all_compounds, reactions,
                                          without_ex, media, testplus,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In the model there are : 1216 compounds and 1584 reactions
We are loading 1216 rows and 1584 columns


The FBA completed with a flux value of 0.0 --> growth: False


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1216
Number of reactions in SM: 1584
Revised number of total reactions: 1758
Number of total compounds: 1553
SMat dimensions: 1216 x 1584


In [20]:
for c in biomass_equation.left_compounds:
    print(f"'{c}' : {biomass_equation.get_left_compound_abundance(c)}")

'cpd00015: FAD (location: c)' : 0.00778132482043096
'cpd00065: L_Tryptophan (location: e)' : 0.0472899299502361
'cpd02229: Bactoprenyl_diphosphate (location: c)' : 0.0609084652443221
'cpd00017: S_Adenosyl_L_methionine (location: c)' : 0.00778132482043096
'cpd00132: L_Asparagine (location: e)' : 0.201205267995816
'cpd10515: Fe2 (location: e)' : 0.00778132482043096
'cpd00023: L_Glutamate (location: e)' : 0.219496655995436
'cpd00010: CoA (location: c)' : 0.00778132482043096
'cpd00041: L_Aspartate (location: e)' : 0.201205267995816
'cpd00030: Mn2 (location: e)' : 0.00778132482043096
'cpd00357: TTP (location: c)' : 0.01468498342018
'cpd00034: Zn2 (location: e)' : 0.00778132482043096
'cpd00058: Cu2 (location: e)' : 0.00778132482043096
'cpd10516: fe3 (location: e)' : 0.00778132482043096
'cpd00156: L_Valine (location: e)' : 0.352889948968272
'cpd17041: Protein_biosynthesis (location: c)' : 1.0
'cpd11493: ACP (location: c)' : 0.00778132482043096
'cpd00016: Pyridoxal_phosphate (location: c)' : 0

In [22]:
status, value, growth = PyFBA.fba.run_fba(all_compounds, reactions,
                                          without_ex, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


In the model there are : 1163 compounds and 1584 reactions
We are loading 1163 rows and 1584 columns


The FBA completed with a flux value of 402.6089941082853 --> growth: True


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1163
Number of reactions in SM: 1584
Revised number of total reactions: 1758
Number of total compounds: 1553
SMat dimensions: 1163 x 1584


In [23]:
biomass_equation.equation

' (0.00778132482043096) cpd00063: Ca2 (location: c) +  (0.352889948968272) cpd00156: L_Valine (location: e) +  (0.00778132482043096) cpd00030: Mn2 (location: e) +  (0.00778132482043096) cpd00205: K (location: c) +  (0.428732289454499) cpd00035: L_Alanine (location: e) +  (0.128039715997337) cpd00060: L_Methionine (location: e) +  (0.15480760087483) cpd00066: L_Phenylalanine (location: c) +  (0.00778132482043096) cpd00017: S_Adenosyl_L_methionine (location: c) +  (0.00778132482043096) cpd00010: CoA (location: c) +  (0.0609084652443221) cpd15665: Peptidoglycan_polymer_n_subunits (location: c) +  (0.0841036156544863) cpd00052: CTP (location: c) +  (0.00778132482043096) cpd10516: fe3 (location: e) +  (0.01468498342018) cpd00357: TTP (location: c) +  (0.00778132482043096) cpd00099: Cl_ (location: e) +  (0.01468498342018) cpd00356: dCTP (location: c) +  (0.00778132482043096) cpd10515: Fe2 (location: e) +  (0.00778132482043096) cpd00254: Mg (location: c) +  (0.242249358141304) cpd00322: L_Iso

In [24]:
uptake_secretion_reactions

{}

In [25]:
filtered_compounds

{<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f470bc00f70>,
 ...}